In [24]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')

import torch as t
import einops
from typing import Dict, List
import torch.sparse as sparse
from tqdm import tqdm
import gc
import torch as t
import torch.sparse as sparse
import einops
from tqdm import tqdm
import pickle
from typing import Dict, List
from pathlib import Path

from trainers.scae import SCAESuite
from buffer import AllActivationBuffer
from utils import load_model_with_folded_ln2, load_iterable_dataset
from find_top_connections import get_importance_scores
from trainers.top_k import AutoEncoderTopK

DTYPE = t.bfloat16
device = t.device('cuda' if t.cuda.is_available() else 'cpu')
t.manual_seed(42)
t.set_grad_enabled(False)
gc.collect()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


1374

In [ ]:
with open("/root/dictionary_learning/connections_100.pkl", "rb") as f:
    connections = pickle.load(f)

In [122]:
model = load_model_with_folded_ln2("gpt2", device=device, torch_dtype=DTYPE)
data = load_iterable_dataset('Skylion007/openwebtext')
suite = SCAESuite.from_pretrained(
    'jacobcd52/gpt2_suite_folded_ln',
    device=device,
    dtype=DTYPE,
    )

initial_submodule = model.transformer.h[0]
layernorm_submodules = {}
submodules = {}
for layer in range(model.config.n_layer):
    submodules[f"mlp_{layer}"] = (model.transformer.h[layer].mlp, "in_and_out")
    submodules[f"attn_{layer}"] = (model.transformer.h[layer].attn, "out")

    layernorm_submodules[f"mlp_{layer}"] = model.transformer.h[layer].ln_2

buffer = AllActivationBuffer(
    data=data,
    model=model,
    submodules=submodules,
    initial_submodule=initial_submodule,
    layernorm_submodules=layernorm_submodules,
    d_submodule=model.config.n_embd,
    n_ctxs=128,
    out_batch_size = 256,
    refresh_batch_size = 256,
    device=device,
    dtype=DTYPE,
)

/root/dictionary_learning/notebooks/../trainers/scae.py:653: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  
/root/dictionary_learning/notebooks/../buffer.py:237: FutureWarni

In [123]:
suite.connections = connections

In [124]:
(initial_acts, input_acts, target_acts, layernorm_scales) = next(buffer)

In [160]:
out = suite.pruned_forward_train(initial_acts, input_acts, layernorm_scales, n_threshold=10, n_random=10)

In [161]:
out['mlp_0']['topk'][0]

tensor([ 6.1875e+00,  1.8500e+01,  4.3125e+00,  3.9219e+00,  5.2500e+00,
         3.5000e+00,  3.4375e+00,  1.6406e+00, -1.5625e-01,  1.9062e+00,
         2.0469e+00,  9.6875e-01,  4.3750e-01,  5.3125e-01,  3.5781e+00,
        -4.2188e-01,  1.2812e+00,  1.2969e+00, -8.7500e-01,  1.1562e+00,
        -1.8750e-01,  1.6250e+00, -5.7812e-01,  7.1094e-01, -3.2812e-01,
         5.9375e-01,  5.6250e-01,  7.5000e-01, -8.2031e-01, -9.3750e-01,
         1.5938e+00,  3.1250e-02, -9.3750e-02, -3.1250e-01, -5.9375e-01,
         7.6562e-01,  1.4844e+00,  1.5625e-02, -2.2461e-01,  6.2500e-01,
         5.2344e-01,  5.4688e-01, -6.2500e-01, -2.1484e-01,  1.4922e+00,
        -9.4727e-02,  6.2500e-02,  1.3281e-01,  3.3594e-01, -7.6562e-01,
        -2.4688e+00,  7.8125e-03, -3.7500e-01,  3.5938e-01,  8.0469e-01,
        -5.4688e-01, -3.1250e-01, -6.5625e-01,  7.8125e-01,  2.6562e-01,
        -8.4375e-01, -2.0312e-01,  2.2969e+00,  9.0625e-01,  2.2344e+00,
        -5.6250e-01, -4.4531e-01, -1.8750e-01, -7.3

In [159]:
out['mlp_0']['topk'][0]

tensor([ 6.1875e+00,  1.8500e+01,  4.3125e+00,  3.9219e+00,  5.2500e+00,
         3.5000e+00,  3.4375e+00,  1.6406e+00, -1.5625e-01,  1.9062e+00,
         2.0469e+00,  9.6875e-01,  4.3750e-01,  5.3125e-01,  3.5781e+00,
        -4.2188e-01,  1.2812e+00,  1.2969e+00, -8.7500e-01,  1.1562e+00,
        -1.8750e-01,  1.6250e+00, -5.7812e-01,  7.1094e-01, -3.2812e-01,
         5.9375e-01,  5.6250e-01,  7.5000e-01, -8.2031e-01, -9.3750e-01,
         1.5938e+00,  3.1250e-02, -9.3750e-02, -3.1250e-01, -5.9375e-01,
         7.6562e-01,  1.4844e+00,  1.5625e-02, -2.2461e-01,  6.2500e-01,
         5.2344e-01,  5.4688e-01, -6.2500e-01, -2.1484e-01,  1.4922e+00,
        -9.4727e-02,  6.2500e-02,  1.3281e-01,  3.3594e-01, -7.6562e-01,
        -2.4688e+00,  7.8125e-03, -3.7500e-01,  3.5938e-01,  8.0469e-01,
        -5.4688e-01, -3.1250e-01, -6.5625e-01,  7.8125e-01,  2.6562e-01,
        -8.4375e-01, -2.0312e-01,  2.2969e+00,  9.0625e-01,  2.2344e+00,
        -5.6250e-01, -4.4531e-01, -1.8750e-01, -7.3

In [150]:
ae = suite.aes['mlp_0']
x = t.randn(1, 768, device=device, dtype=DTYPE)

In [153]:
out = ae.encode(x, n_threshold=10, return_topk=True)

In [152]:
out[1]

tensor([[31.8750, 27.0000, 24.3750, 23.2500, 22.2500, 21.1250, 19.2500, 19.1250,
         19.1250, 18.8750, 18.5000, 18.5000, 18.2500, 18.2500, 17.8750, 17.8750,
         17.6250, 17.3750, 17.2500, 17.0000, 17.0000, 16.8750, 16.8750, 16.7500,
         16.6250, 16.6250, 16.6250, 16.6250, 16.5000, 16.3750, 16.2500, 16.2500,
         16.2500, 16.1250, 16.0000, 15.9375, 15.9375, 15.8750, 15.8125, 15.7500,
         15.7500, 15.6875, 15.6875, 15.6250, 15.6250, 15.4375, 15.2500, 15.1875,
         15.1875, 15.1875, 15.1250, 15.0625, 15.0625, 15.0625, 14.9375, 14.9375,
         14.8125, 14.8125, 14.6250, 14.4375, 14.3750, 14.3125, 14.2500, 14.1250,
         14.0000, 14.0000, 13.9375, 13.9375, 13.9375, 13.8750, 13.8750, 13.8750,
         13.6875, 13.6875, 13.6250, 13.5625, 13.5000, 13.4375, 13.3750, 13.3125,
         13.3125, 13.3125, 13.2500, 13.2500, 13.1250, 13.0625, 13.0625, 13.0000,
         12.9375, 12.9375, 12.9375, 12.9375, 12.8750, 12.8750, 12.8750, 12.8125,
         12.8125, 12.8125, 1

In [154]:
out[1]

tensor([[31.8750, 27.0000, 24.3750, 23.2500, 22.2500, 21.1250, 19.2500, 19.1250,
         19.1250, 18.8750, 18.5000, 18.5000, 18.2500, 18.2500, 17.8750, 17.8750,
         17.6250, 17.3750, 17.2500, 17.0000, 17.0000, 16.8750, 16.8750, 16.7500,
         16.6250, 16.6250, 16.6250, 16.6250, 16.5000, 16.3750, 16.2500, 16.2500,
         16.2500, 16.1250, 16.0000, 15.9375, 15.9375, 15.8750, 15.8125, 15.7500,
         15.7500, 15.6875, 15.6875, 15.6250, 15.6250, 15.4375, 15.2500, 15.1875,
         15.1875, 15.1875, 15.1250, 15.0625, 15.0625, 15.0625, 14.9375, 14.9375,
         14.8125, 14.8125, 14.6250, 14.4375, 14.3750, 14.3125, 14.2500, 14.1250,
         14.0000, 14.0000, 13.9375, 13.9375, 13.9375, 13.8750, 13.8750, 13.8750,
         13.6875, 13.6875, 13.6250, 13.5625, 13.5000, 13.4375, 13.3750, 13.3125,
         13.3125, 13.3125, 13.2500, 13.2500, 13.1250, 13.0625, 13.0625, 13.0000,
         12.9375, 12.9375, 12.9375, 12.9375, 12.8750, 12.8750, 12.8750, 12.8125,
         12.8125, 12.8125, 1